In [143]:
import requests as req
from bs4 import BeautifulSoup as bs

# 操作 browser 的 驅動程式
from selenium import webdriver

# 負責開啟和關閉 Chrome 的套件
from selenium.webdriver.chrome.service import Service

# 自動下載 Chrome Driver 的套件
from webdriver_manager.chrome import ChromeDriverManager

# 啟動瀏覽器工具的選項
from selenium.webdriver.chrome.options import Options

# 例外處理的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待、了解某個元素的狀態，通常與 exptected_conditions 和 By 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，經常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 隨機取得 User-Agent
from fake_useragent import UserAgent
# ua = UserAgent()

import pymysql
from pymysql.err import IntegrityError
from time import sleep
import json
import random
import os
from datetime import datetime
from datetime import timedelta
import re
from pprint import pprint
from collections import Counter

In [144]:
user_agent_list = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; AS; rv:11.0) like Gecko",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.79 Safari/537.36 Edge/14.14393",
    "Mozilla/5.0 (Linux; Android 6.0.1; Nexus 6P Build/MMB29P) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.83 Mobile Safari/537.36"
    "Mozilla/5.0 (iPhone14,3; U; CPU iPhone OS 15_0 like Mac OS X) AppleWebKit/602.1.50 (KHTML, like Gecko) Version/10.0 Mobile/19A346 Safari/602.1"
    # Add more user agent strings as needed
]
user_agent = random.choice(user_agent_list)

my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")                #不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")  # 最大化視窗
my_options.add_argument("--incognito")  # 開啟無痕模式
my_options.add_argument("--disable-popup-blocking")  # 禁用彈出攔截
my_options.add_argument("--disable-notifications")  # 取消 chrome 推播通知
my_options.add_argument("--lang=zh-TW")  # 設定為正體中文
my_options.add_argument('blink-settings=imagesEnabled=false')  # 不載入圖
my_options.add_argument(f'--user-agent={user_agent}')  # 隨機 user-angent 
my_options.add_experimental_option(
    "excludeSwitches", ['enable-automation', 'enable-logging'])  # 沒有異常log


driver = webdriver.Chrome(
    options=my_options,
    service=Service(ChromeDriverManager().install()))

In [145]:
# 建立下載路徑/資料夾，不存在就新增 (os.getcwd() 會取得當前的程式工作目錄)
folderPath = os.path.join(os.getcwd(), 'crawler_data')
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

#預設下載路徑
my_options.add_experimental_option("prefs", {
  "download.default_directory": folderPath
})

In [146]:
url = "http://jsjustweb.jihsun.com.tw/z/zh/zha/zha.djhtm"


results_v1 = []
results_v2 = []

alinks = []
alinks_list = [] # final sub grp links

main_grp = []
main_grp_list = [] # main grp stocks list

alinks_list_main = [] # final main grp links




In [147]:
def visit():
    driver.get(url)

In [148]:
# sub grp links
def inputlink_sub_grp():

    fields = driver.find_elements(By.CSS_SELECTOR, "td.t3t1 > a")
    fields_rev = driver.find_elements(By.CSS_SELECTOR, "td.t3t1_rev > a")

    # extract total links
    for index, i in enumerate(fields):
        grp = i.get_attribute("innerText")
        grp_link = i.get_attribute("href")
        # print(grp, grp_link)

        link_dict = {
            "grp_name" : grp,
            "grp_link" : grp_link,
        }

        alinks.append(link_dict)


    for index, j in enumerate(fields_rev):
        grp_rev = j.get_attribute("innerText")
        grp_link_rev= j.get_attribute("href")
        # print(grp_rev, grp_link_rev)

        link_dict = {
            "grp_name" : grp_rev,
            "grp_link" : grp_link_rev,
        }

        alinks.append(link_dict)

    # final links
    unique_alinks = set(frozenset(d.items()) for d in alinks)

    for d in unique_alinks:
        alinks_list.append(dict(d))



In [149]:
def main_stocks_grp():
    global main_grp
    global main_grp_list
    
    for i in alinks:
        main_grp.append(list(i.values())[0])

    main_grp = list(Counter(main_grp).items())

    for index, i in enumerate(main_grp):
        if main_grp[index][1] > 1:
            main_grp_list.append(main_grp[index][0])

In [150]:
def inputlink_main_grp():

    main_stocks_grp()
    
    global alinks_list_main
    
    for i in main_grp_list:
        for j in alinks_list:
            if i == j['grp_name']:
                links_dict = {
                    'grp_name' : j['grp_name'],
                    'grp_link' : j['grp_link']
                }
                alinks_list_main.append(links_dict)
    
    # print(alinks_list_main)
    
    

In [151]:
def stocks_list_v1():
    # stocks_dict = {}
    global results_v1
    results_v1 = []
    n = 0
    
    for i in alinks_list_main:
        
        # global = stocks_dict
        grp_name = i['grp_name']
        grp_link = i['grp_link']

        # print(grp_name, grp_link)

        response = req.get(grp_link)
        html_content = response.content
        soup = bs(html_content, 'html.parser')


        for i in range(len(soup.select(('td.t3t1')))):
            table = soup.select('td.t3t1')[i]
            stock = table.select('script[language="javascript"]')[0].get_text()

            pattern = r"'.*?'"
            match = re.findall(pattern, stock)

            # print(match[0][3:-1])

            stocks_dict = {
                "category" : grp_name,
                "code" : match[0][3:-1],
                "name" : match[1][1:-1]
            }

            results_v1.append(stocks_dict)
            
            n += 1
            # if n % 300 == 0:
            #     print(n)


        for i in range(len(soup.select(('td.t3t1_rev')))):
            table = soup.select('td.t3t1_rev')[i]
            stock = table.select('script[language="javascript"]')[0].get_text()

            pattern = r"'.*?'"
            match = re.findall(pattern, stock)

            # print(match)

            stocks_dict = {
                "category" : grp_name,
                "code" : match[0][3:-1],
                "name" : match[1][1:-1]
            }
            
            results_v1.append(stocks_dict)
    
            n += 1
            # if n % 300 == 0:
            #     print(n)

    

In [152]:
alinks_list

[]

In [153]:
def stocks_list_v2():
    # stocks_dict = {}
    global results_v2
    results_v2 = []
    n = 0
    
    for i in alinks_list:
        
        # global = stocks_dict
        grp_name = i['grp_name']
        grp_link = i['grp_link']

        # print(grp_name, grp_link)

        response = req.get(grp_link)
        html_content = response.content
        soup = bs(html_content, 'html.parser')


        for i in range(len(soup.select(('td.t3t1')))):
            table = soup.select('td.t3t1')[i]
            stock = table.select('script[language="javascript"]')[0].get_text()

            pattern = r"'.*?'"
            match = re.findall(pattern, stock)

            # print(match[0][3:-1])

            stocks_dict = {
                "category" : grp_name,
                "code" : match[0][3:-1],
                "name" : match[1][1:-1]
            }

            results_v2.append(stocks_dict)
            
            n += 1
            # if n % 300 == 0:
            #     print(n)


        for i in range(len(soup.select(('td.t3t1_rev')))):
            table = soup.select('td.t3t1_rev')[i]
            stock = table.select('script[language="javascript"]')[0].get_text()

            pattern = r"'.*?'"
            match = re.findall(pattern, stock)

            # print(match)

            stocks_dict = {
                "category" : grp_name,
                "code" : match[0][3:-1],
                "name" : match[1][1:-1]
            }
            
            results_v2.append(stocks_dict)
    
            n += 1
            # if n % 300 == 0:
            #     print(n)


In [154]:
def integration_stocks_list():
    for j in results_v1:
        j["sub_category"] = []

    n = 0

    for i in results_v2:
        for j in results_v1:
            if j['name'] == i['name']:
                j['sub_category'].append(i['category'])
                # print(j)
        n += 1
        if n % 100 == 0:
            print(f"第{n}筆新增完畢")

In [155]:
if __name__ == "__main__":
    visit()
    inputlink_sub_grp()
    inputlink_main_grp()
    stocks_list_v1()
    stocks_list_v2()
    integration_stocks_list()
    
    # alinks_list
    # alinks_list_main

        


第100筆新增完畢
第200筆新增完畢
第300筆新增完畢
第400筆新增完畢
第500筆新增完畢
第600筆新增完畢
第700筆新增完畢
第800筆新增完畢
第900筆新增完畢
第1000筆新增完畢
第1100筆新增完畢
第1200筆新增完畢
第1300筆新增完畢
第1400筆新增完畢
第1500筆新增完畢
第1600筆新增完畢
第1700筆新增完畢
第1800筆新增完畢
第1900筆新增完畢
第2000筆新增完畢
第2100筆新增完畢
第2200筆新增完畢
第2300筆新增完畢
第2400筆新增完畢
第2500筆新增完畢
第2600筆新增完畢
第2700筆新增完畢
第2800筆新增完畢
第2900筆新增完畢
第3000筆新增完畢
第3100筆新增完畢
第3200筆新增完畢
第3300筆新增完畢
第3400筆新增完畢
第3500筆新增完畢
第3600筆新增完畢
第3700筆新增完畢
第3800筆新增完畢
第3900筆新增完畢
第4000筆新增完畢
第4100筆新增完畢
第4200筆新增完畢
第4300筆新增完畢
第4400筆新增完畢
第4500筆新增完畢
第4600筆新增完畢
第4700筆新增完畢
第4800筆新增完畢
第4900筆新增完畢
第5000筆新增完畢
第5100筆新增完畢
第5200筆新增完畢
第5300筆新增完畢
第5400筆新增完畢
第5500筆新增完畢
第5600筆新增完畢
第5700筆新增完畢
第5800筆新增完畢
第5900筆新增完畢
第6000筆新增完畢
第6100筆新增完畢
第6200筆新增完畢
第6300筆新增完畢


In [156]:
# results_v1

# for i in results_v2:
#     for j in results_v1:
#         if j['name'] == i['name']:
#             j['sub_category'].append(i['category'])
#             print(j)

In [157]:
results_v1

[{'category': '水泥',
  'code': '1101',
  'name': '台泥',
  'sub_category': ['汽電共生', '水泥', '電力', '預拌混凝土', '水泥製品']},
 {'category': '水泥',
  'code': '1103',
  'name': '嘉泥',
  'sub_category': ['物業投資發展', '水泥', '運輸事業', '地產', '倉儲／貨櫃場']},
 {'category': '水泥',
  'code': '1108',
  'name': '幸福',
  'sub_category': ['水泥', '預拌混凝土', '高爐水泥']},
 {'category': '水泥',
  'code': '1110',
  'name': '東泥',
  'sub_category': ['水泥', '高爐水泥']},
 {'category': '水泥',
  'code': '5520',
  'name': '力泰',
  'sub_category': ['水泥', '預拌混凝土']},
 {'category': '水泥',
  'code': '8463',
  'name': '潤泰材',
  'sub_category': ['營造工程', '建築設計、營繕裝璜', '水泥', '高爐水泥']},
 {'category': '水泥',
  'code': '9930',
  'name': '中聯資源',
  'sub_category': ['傳產其他', '水泥', '回收、焚化相關', '高爐水泥']},
 {'category': '水泥',
  'code': '1102',
  'name': '亞泥',
  'sub_category': ['汽電共生', '水泥', '電力', '預拌混凝土', '高爐水泥']},
 {'category': '水泥',
  'code': '1104',
  'name': '環泥',
  'sub_category': ['水泥', '預拌混凝土', '水泥製品']},
 {'category': '水泥',
  'code': '1109',
  'name': '信大',
  'sub_cate